# Set-up

In [1]:
import os
import numpy as np
import seqdata as sd
import xarray as xr

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# Set wd
os.chdir("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq")

# Set output dir
outdir_path = "training/2023_12_20/bp_res"

# Make output dir if it doesn't exist
if not os.path.exists(outdir_path):
    os.makedirs(outdir_path)

# Create a bed file

In [3]:
# Original SeqData with regions
sdata_orig = sd.open_zarr("training/2023_12_19/seqdatasets/deAlmeida22_training.zarr")
sdata_orig

<xarray.Dataset>
Dimensions:                                  (_sequence: 442866, _ohe: 4,
                                              length: 249, _length: 249)
Dimensions without coordinates: _sequence, _ohe, length, _length
Data variables: (12/18)
    Dev_log2_enrichment                      (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Dev_log2_enrichment_scaled               (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Hk_log2_enrichment                       (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Hk_log2_enrichment_quantile_normalized   (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    Hk_log2_enrichment_scaled                (_sequence) float64 dask.array<chunksize=(55359,), meta=np.ndarray>
    ...                                       ...
    seq                                      (_sequence, _length) |S1 dask.array<chunksize=(10000, 249), meta=np.ndarray>
    set                                      (_sequence) <U8 dask.array<chunksize=(13840,), meta=np.ndarray>
    start                                    (_sequence) <U8 dask.array<chunksize=(13840,), meta=np.ndarray>
    strand                                   (_sequence) <U1 dask.array<chunksize=(110717,), meta=np.ndarray>
    train_val                                (_sequence) bool dask.array<chunksize=(221433,), meta=np.ndarray>
    type                                     (_sequence) <U9 dask.array<chunksize=(13840,), meta=np.ndarray>

In [4]:
# Save just the chrom, start, end as bed file
training_bed = sdata_orig[["chrom", "start", "end"]].to_dataframe()
training_bed.to_csv(os.path.join(outdir_path, "training.bed"), sep="\t", index=False, header=False)

# Build SeqData

In [5]:
# Input files
fasta_file = "/cellar/users/aklie/data/ref/genomes/dm3/dm3.fa"
bed_file = os.path.join(outdir_path, "training.bed")
signal_files = ['analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.bw', 'analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.bw']
input_files = ['analysis/2023_12_11/geo/GSM5574549_S2_dev_STARRseq_input.bw', 'analysis/2023_12_11/geo/GSM5574552_S2_hk_STARRseq_input.bw']
bigwigs = signal_files + input_files
bigwigs

['analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.bw',
 'analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.bw',
 'analysis/2023_12_11/geo/GSM5574549_S2_dev_STARRseq_input.bw',
 'analysis/2023_12_11/geo/GSM5574552_S2_hk_STARRseq_input.bw']

In [6]:
# Make sure all bigwigs exist
for bw in bigwigs:
    assert os.path.exists(bw), f"{bw} does not exist"

In [7]:
# Compose a SeqData from a set of files - only run this once!
sdata = sd.from_region_files(
    sd.GenomeFASTA(
        'seq',
        fasta_file,
        batch_size=2048,
        n_threads=4,
    ),
    sd.BigWig(
        'cov',
        bigwigs,
        ['dev_signal', 'hk_signal', 'dev_input', 'hk_input'],
        batch_size=2048,
        n_jobs=4,
        threads_per_job=2,
    ),
    path=os.path.join(outdir_path, "deepstarr_training_bpres.zarr"),
    fixed_length=2114,
    bed=bed_file,
    overwrite=True,
    max_jitter=128
)

100%|██████████| 442866/442866 [00:14<00:00, 29925.92it/s]
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
100%|██████████| 442866/442866 [01:24<00:00, 5219.49it/s]


In [8]:
# Add in original train_val split
sdata["train_val"] = sdata_orig["train_val"]
sdata["train_val"].to_series().value_counts()

True     402296
False     40570
Name: train_val, dtype: int64

In [9]:
# Need to upper case the seqs for ohe
sdata["cleaned_seq"] = xr.DataArray(np.char.upper(sdata["seq"]), dims=["_sequence", "_length"])

In [10]:
# Check how many b'N' characters exist in "cleaned_seq"
(sdata["cleaned_seq"] == b"N").sum().values

array(203080)

In [11]:
sdata["cov"][0][0].values.sum(), sdata["cov"][0][1].values.sum(), sdata["cov"][0][2].values.sum(), sdata["cov"][0][3].values.sum()

(17461, 5535, 1469, 3300)

In [12]:
# Split cov into hk and dev
sdata['dev_signal'] = (
    sdata.cov.sel(cov_sample=['dev_signal'])
    .drop_vars("cov_sample").squeeze()
)
sdata['hk_signal'] = (
    sdata.cov.sel(cov_sample=['hk_signal'])
    .drop_vars("cov_sample").squeeze()
)
sdata['dev_input'] = (
    sdata.cov.sel(cov_sample=['dev_input'])
    .drop_vars("cov_sample").squeeze()
)
sdata['hk_input'] = (
    sdata.cov.sel(cov_sample=['hk_input'])
    .drop_vars("cov_sample").squeeze()
)

In [13]:
sdata["dev_signal"][0].values.sum(), sdata["hk_signal"][0].values.sum(), sdata["dev_input"][0].values.sum(), sdata["hk_input"][0].values.sum()

(17461, 5535, 1469, 3300)

In [14]:
# Get rid of aggregated cov
sdata = sdata.drop_vars("cov")

In [15]:
# Make things str
sdata["chrom"] = sdata["chrom"].astype(str)
sdata["strand"] = sdata["strand"].astype(str)
sdata["cov_sample"] = sdata["cov_sample"].astype(str)

In [16]:
sdata

<xarray.Dataset>
Dimensions:      (_sequence: 442866, cov_sample: 4, _length: 2370)
Coordinates:
  * cov_sample   (cov_sample) <U10 'dev_signal' 'hk_signal' ... 'hk_input'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom        (_sequence) <U1 dask.array<chunksize=(55359,), meta=np.ndarray>
    chromEnd     (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    chromStart   (_sequence) int64 dask.array<chunksize=(55359,), meta=np.ndarray>
    seq          (_sequence, _length) |S1 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    strand       (_sequence) <U1 dask.array<chunksize=(55359,), meta=np.ndarray>
    train_val    (_sequence) bool dask.array<chunksize=(221433,), meta=np.ndarray>
    cleaned_seq  (_sequence, _length) |S1 b'C' b'T' b'C' b'T' ... b'A' b'G' b'C'
    dev_signal   (_sequence, _length) uint16 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    hk_signal    (_sequence, _length) uint16 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    dev_input    (_sequence, _length) uint16 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
    hk_input     (_sequence, _length) uint16 dask.array<chunksize=(2048, 2370), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    128
    sequence_dim:  _sequence

In [17]:
# Save to Zarr
sd.to_zarr(sdata, os.path.join(outdir_path, "deepstarr_training_bpres_processed.zarr"), mode="w")

# DONE!

---